
Circuit Simulation Example with OBI-One
======================================

This notebook demonstrates how to run circuit simulations using the obi-one simulation framework
with [BlueCelluLab](https://github.com/openbraininstitute/BlueCelluLab) as the simulator. Let's get the path to the simulation configuration and circuit configuration file paths and the population name of the circuit that you want to simulate.

For this example, we will use the pre-made `simulation_config.json` (simulation configuration) file instead of the one generated by obi-one.

To Do
- get circuit from entioty core
- generate simualtion config
- stage if necessary
- run it with bluecellulab

Get circuit from entitycore

In [5]:
from entitysdk import Client, ProjectContext, models
from obi_auth import get_token
import os
import time
import obi_one as obi
from pathlib import Path

# Authenticate
proj_url = "https://www.openbraininstitute.org/app/virtual-lab/lab/04c93af1-5955-43a9-9180-b8a4bda4712c/project/50280a36-44b9-45e7-8b7d-db220a203ce5/explore/interactive/model/circuit?br_id=4642cddb-4fbe-4aae-bbf7-0946d6ada066&br_av=8"
token = get_token(environment="production", auth_mode="daf")
project_context = ProjectContext.from_vlab_url(proj_url)
client = Client(environment="production", project_context=project_context, token_manager=token)

In [2]:
# Download using unique ID
# nbS1-O1-E2PV-maxNsyn-HEX0-L3
entity_ID = "b595c41d-b721-4125-8048-a6e2164cdd08"  # <<< FILL IN UNIQUE CIRCUIT ID HERE

# # Fetch circuit
fetched = client.get_entity(entity_id=entity_ID, entity_type=models.Circuit)
print(f"Circuit fetched: {fetched.name} (ID {fetched.id})\n")
print(f"#Neurons: {fetched.number_neurons}, #Synapses: {fetched.number_synapses}, #Connections: {fetched.number_connections}\n")
print(f"{fetched.description}\n")

# Download SONATA circuit files
asset = [asset for asset in fetched.assets if asset.label=="sonata_circuit"][0]
asset_dir = asset.path 
circuit_dir = "analysis_circuit"
assert not os.path.exists(asset_dir), f"ERROR: Circuit download folder '{asset_dir}' already exists! Please delete folder."
assert not os.path.exists(circuit_dir), f"ERROR: Circuit folder '{circuit_dir}' already exists! Delete folder or choose a different path."

t0 = time.time()
client.download_directory(
    entity_id=fetched.id,
    entity_type=models.Circuit,
    asset_id=asset.id,
    output_path=".",
    max_concurrent=4,  # Parallel file download
)
t = time.time() - t0
print(f"Circuit files downloaded to '{asset_dir}' in {t:.1f}s")

2025-07-23 15:17:32,444 - httpx - INFO - HTTP Request: GET https://www.openbraininstitute.org/api/entitycore/circuit/b595c41d-b721-4125-8048-a6e2164cdd08 "HTTP/1.1 200 OK"
Circuit fetched: nbS1-O1-E2PV-maxNsyn-HEX0-L3 (ID b595c41d-b721-4125-8048-a6e2164cdd08)

#Neurons: 2, #Synapses: 48, #Connections: 1

A neuron pair from the nbS1-O1 circuit, located in layer 3 of subcolumn HEX0. The pair consists of one excitatory neuron targeting one inhibitory PV (parvalbumin) neuron, with the highest number of synapses connecting them.

2025-07-23 15:17:32,624 - httpx - INFO - HTTP Request: GET https://www.openbraininstitute.org/api/entitycore/circuit/b595c41d-b721-4125-8048-a6e2164cdd08/assets/654a7853-1720-483b-8108-edef8c146b05 "HTTP/1.1 200 OK"
2025-07-23 15:17:32,959 - httpx - INFO - HTTP Request: GET https://www.openbraininstitute.org/api/entitycore/circuit/b595c41d-b721-4125-8048-a6e2164cdd08/assets/654a7853-1720-483b-8108-edef8c146b05/list "HTTP/1.1 200 OK"
2025-07-23 15:17:33,247 - httpx 

Generate Simulation Config with new nodeset

In [8]:
circuit_path_prefix = "sonata_circuit/"
circ_path = f"{circuit_path_prefix}/circuit_config.json"
circuit = obi.Circuit(name="nbS1-O1-E2PV-maxNsyn-HEX0-L3", path=circ_path)
print(f"Circuit '{circuit}' with {circuit.sonata_circuit.nodes.size} neurons and {circuit.sonata_circuit.edges.size} synapses")
print(f"Default node population: '{circuit.default_population_name}'")

Circuit 'nbS1-O1-E2PV-maxNsyn-HEX0-L3' with 720 neurons and 3112 synapses
Default node population: 'S1nonbarrel_neurons'


In [ ]:
# Get SONATA circuit object
c = circuit.sonata_circuit


...id': [], 'layer': ['6']}}


In [ ]:
# Adding a node set to the circuit
obi.NeuronSet.add_node_set_to_circuit(c, {"my_new_node_set": {
    "population": "S1nonbarrel_neurons",
    "node_id": [0, 1]
  }
})

In [ ]:
print("..." + str(c.node_sets.content)[-80:])

{'Mosaic': ['All'],
 'All': ['L4_SBC',
  'L5_TPC:B',
  'L4_NBC',
  'L4_DBC',
  'L23_DBC',
  'L5_TPC:A',
  'L3_TPC:A',
  'L4_BTC',
  'L6_BTC',
  'L4_UPC',
  'L6_LBC',
  'L6_IPC',
  'L23_BP',
  'L4_MC',
  'L5_DBC',
  'L6_NBC',
  'L1_LAC',
  'L6_CHC',
  'L23_MC',
  'L1_HAC',
  'L6_NGC',
  'L4_NGC',
  'L2_IPC',
  'L6_HPC',
  'L5_BTC',
  'L4_CHC',
  'L5_NGC',
  'L4_BP',
  'L2_TPC:A',
  'L5_SBC',
  'L5_UPC',
  'L6_DBC',
  'L23_CHC',
  'L5_NBC',
  'L1_NGC-SA',
  'L6_BPC',
  'L6_UPC',
  'L23_LBC',
  'L4_SSC',
  'L5_LBC',
  'L6_SBC',
  'L1_NGC-DA',
  'L6_TPC:C',
  'L23_SBC',
  'L5_MC',
  'L1_DAC',
  'L2_TPC:B',
  'L6_TPC:A',
  'L5_CHC',
  'L23_NBC',
  'L3_TPC:C',
  'L6_BP',
  'L4_LBC',
  'L5_BP',
  'L23_BTC',
  'L6_MC',
  'L1_SAC',
  'L4_TPC',
  'L5_TPC:C',
  'L23_NGC'],
 'Excitatory': {'synapse_class': 'EXC'},
 'Inhibitory': {'synapse_class': 'INH'},
 'L1_DAC': {'mtype': 'L1_DAC'},
 'L1_HAC': {'mtype': 'L1_HAC'},
 'L1_LAC': {'mtype': 'L1_LAC'},
 'L1_NGC-DA': {'mtype': 'L1_NGC-DA'},
 'L1_NGC-SA

In [21]:
# Write new circuit's node set file
obi.NeuronSet.write_circuit_node_set_file(c, output_path="./", file_name="new_node_sets.json", overwrite_if_exists=True)

In [24]:
# Sim duration
sim_duration = 3000.0

# Empty Simulation Configuration
sim_conf = obi.SimulationsForm.empty_config()

# Info
info = obi.Info(campaign_name="Small Microcircuit Simulation", campaign_description="Simulation of circuit with predefined neuron set and constant current stimulus")
sim_conf.set(info, name="info")

# Neuron Sets
sim_neuron_set = obi.IDNeuronSet(neuron_ids=obi.NamedTuple(name="IDNeuronSet1", elements=range(1)))
sim_conf.add(sim_neuron_set, name='ID1')

sync_neuron_set = obi.IDNeuronSet(neuron_ids=obi.NamedTuple(name="IDNeuronSet2", elements=range(1,3)))
sim_conf.add(sync_neuron_set, name='ID2')


# Regular Timesteps
regular_timestamps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=3, interval=sim_duration)
sim_conf.add(regular_timestamps, name='RegularTimestamps')

# Stimulus
poisson_input = obi.PoissonSpikeStimulus(duration=800.0, timestamps=regular_timestamps.ref, frequency=20, source_neuron_set=sim_neuron_set.ref, targeted_neuron_set=sim_neuron_set.ref)
sim_conf.add(poisson_input, name='PoissonInputStimulus')

# sync_input = obi.FullySynchronousSpikeStimulus(timestamps=regular_timestamps.ref, source_neuron_set=sync_neuron_set.ref, targeted_neuron_set=sim_neuron_set.ref)
# sim_conf.add(sync_input, name='SynchronousInputStimulus')

# current_stim = obi.ConstantCurrentClampSomaticStimulus(timestamps=regular_timestamps.ref, neuron_set=sim_neuron_set.ref, amplitude=0.1)
# sim_conf.add(current_stim, name='ConstantCurrentClampSomaticStimulus')

# Recordings
voltage_recording = obi.SomaVoltageRecording(neuron_set=sim_neuron_set.ref)
sim_conf.add(voltage_recording, name='VoltageRecording')

time_window_voltage_recording = obi.TimeWindowSomaVoltageRecording(neuron_set=sim_neuron_set.ref, start_time=0.0, end_time=2000.0)
sim_conf.add(time_window_voltage_recording, name='TimeWindowVoltageRecording')

# Initialization
simulations_initialize = obi.SimulationsForm.Initialize(circuit=obi.CircuitFromID(id_str=entity_ID), 
                                                        node_set=sim_neuron_set.ref, 
                                                        simulation_length=sim_duration)
sim_conf.set(simulations_initialize, name='initialize')

# Validated Config
validated_sim_conf = sim_conf.validated_config()

In [ ]:
population_name = "S1nonbarrel_neurons"
circuit_folder = "../data/tiny_circuits/N_10__top_nodes_dim6__asc/"
circuit_config_path = "../data/tiny_circuits/N_10__top_nodes_dim6__asc/circuit_config.json"
simulation_config_path = "../data/tiny_circuits/N_10__top_nodes_dim6__asc/simulation_config.json"

Get the path to the mod files and compile the mod files. 

In [ ]:
# Remove the old compiled mod files folder
! rm -r arm64/
# flag DISABLE_REPORTINGLIB to skip SonataReportHelper.mod and SonataReport.mod from compilation.
!nrnivmodl -incflags "-DDISABLE_REPORTINGLIB" {circuit_folder}/mod
# !nrnivmodl {circuit_folder}/mod

Import required modules from obi-one

In [ ]:

from obi_one.scientific.simulation.simulations import Simulation
from obi_one.scientific.simulation.stimulus import ConstantCurrentClampSomaticStimulus
from obi_one.scientific.simulation.recording import SomaVoltageRecording
from obi_one.scientific.simulation.timestamps import RegularTimestamps
from obi_one.scientific.circuit.neuron_sets import IDNeuronSet
from obi_one.core.tuple import NamedTuple
from obi_one.scientific.unions.unions_neuron_sets import NeuronSetReference
from obi_one.scientific.unions.unions_timestamps import TimestampsReference
from matplotlib import pyplot as plt

Create the `Simulation` object.

In [ ]:
# Stimulus
poisson_input = obi.PoissonSpikeStimulus(duration=800.0, timestamps=regular_timestamps.ref, frequency=20, source_neuron_set=sim_neuron_set.ref, targeted_neuron_set=sim_neuron_set.ref)
sim_conf.add(poisson_input, name='PoissonInputStimulus')

In [ ]:
# Create regular timestamps with start, end, and dt
timestamps = RegularTimestamps(
    number_of_repetitions=1,
    interval=100.0,  # ms
    start_time=0.0,  # ms
    end_time=100.0,  # ms
    dt=0.1,          # ms
    simulation_level_name="timestamps_1"
)

# Create a NamedTuple with the neuron IDs
neuron_ids = NamedTuple(name="neuron_ids", elements=[1, 2, 3])  # List of cell IDs to include

# Create neuron set using IDNeuronSet
neuron_set = IDNeuronSet(
    population=population_name,
    neuron_ids=neuron_ids,
    simulation_level_name="neuron_set_1"
)

# First, create the timestamps reference
timestamps_ref = TimestampsReference(block=timestamps, block_name="timestamps_1")

# Then create the stimulus with the reference
stimulus = ConstantCurrentClampSomaticStimulus(
    timestamps=timestamps_ref,
    delay=10.0,  # ms
    duration=50.0,  # ms
    amplitude=0.1,  # nA
    neuron_set=NeuronSetReference(block=neuron_set, block_name="neuron_set_1"),
    simulation_level_name="stimulus_1"
)
# Create recordings (example with voltage recording)
recording = SomaVoltageRecording(
    start_time=0.0,
    end_time=100.0,
    dt=0.1,
    neuron_set=NeuronSetReference(block=neuron_set, block_name="neuron_set_1"),
    simulation_level_name="voltage_recording_1"
)

from obi_one.scientific.circuit.circuit import Circuit

# First create the circuit object
circuit = Circuit(
    name="N_10__top_nodes_dim6__asc",
    path=circuit_config_path,
    node_population=population_name,
)

from obi_one.core.info import Info
# Create info object
simulation_info = Info(
    campaign_name="N_10__top_nodes_dim6__asc Simulation",
    campaign_description="A test simulation with a small microcircuit",
)

# Then use it in the simulation initialization
simulation = Simulation(
    name="simulation_1",
    info=simulation_info,
    timestamps={"timestamps_1": timestamps},  # Make sure this matches the block_name
    stimuli={"stimulus_1": stimulus},
    recordings={"voltage_recording_1": recording},
    neuron_sets={"neuron_set_1": neuron_set},
    initialize={
        "circuit": circuit,
        "simulation_length": 100.0,  # ms
        "node_set": NeuronSetReference(block=neuron_set, block_name="neuron_set_1"),
        "timestep": 0.025  # ms
    }
)

In [ ]:
# Get SONATA circuit object
c = circuit.sonata_circuit
print("..." + str(c.node_sets.content)[-25:])

# Adding a node set to the circuit
obi.NeuronSet.add_node_set_to_circuit(c, {"Layer23": {"layer": [2, 3]}})
print("..." + str(c.node_sets.content)[-55:])

# Adding a node set with an exising name => NOT POSSIBLE
# obi.NeuronSet.add_node_set_to_circuit(c, {"Layer23": {"layer": [2, 3]}})  # AssertionError: Node set 'Layer23' already exists!

# Update/overwrite an existing node set
obi.NeuronSet.add_node_set_to_circuit(c, {"Layer23": ["Layer2", "Layer3"]}, overwrite_if_exists=True)  # Update/overwrite
print("..." + str(c.node_sets.content)[-58:])

# Adding multiple node sets
obi.NeuronSet.add_node_set_to_circuit(c, {"Layer45": ["Layer4", "Layer5"], "Layer56": ["Layer5", "Layer6"]})
print("..." + str(c.node_sets.content)[-124:])

# Add node set from NeuronSet object, resolved in circuit's default node population
neuron_set = obi.CombinedNeuronSet(node_sets=("Layer1", "Layer2", "Layer3"))
obi.NeuronSet.add_node_set_to_circuit(c, {"Layer123": neuron_set.get_node_set_definition(circuit, circuit.default_population_name)})
print("..." + str(c.node_sets.content)[-168:])

# Adding a node sets based on previously added node sets
obi.NeuronSet.add_node_set_to_circuit(c, {"AllLayers": ["Layer123", "Layer4", "Layer56"]})
print("..." + str(c.node_sets.content)[-216:])

# Write new circuit's node set file
obi.NeuronSet.write_circuit_node_set_file(c, output_path="./", file_name="new_node_sets.json", overwrite_if_exists=True)

Run circuit simulation using BlueCelluLab backend. In future, we will support Neurodamus backend as well. This will run a SONATA simulation and save voltage traces for the specified cells.  

In [ ]:
# Run the simulation
simulation.run(
    simulation_config=simulation_config_path,
    simulator="bluecellulab", # Optional: bluecellulab or neurodamus. Default: bluecellulab
    save_nwb=True            # Optional: Save results in NWB format. Default: False
)

The results are stored in the output directory of the circuit folder. The logs are also stored in the logs in notebooks folder.

### Spike Report Analysis using BluePySnap

We will use the example [notebook](https://github.com/openbraininstitute/snap/blob/master/doc/source/notebooks/06_spike_reports.ipynb) from BluePySnap to analyse the spike report generated by the circuit simulation.

In [ ]:
import bluepysnap

snap_simulation = bluepysnap.Simulation(simulation_config_path)
spikes = snap_simulation.spikes
print(
    spikes.time_start,
    spikes.time_stop,
    spikes.dt
)
print(spikes.population_names)

See what simulation_config.json file contents:

In [ ]:
snap_simulation.config

In [ ]:
spike_pop = spikes['S1nonbarrel_neurons']
print(type(spike_pop))

node_population = spike_pop.nodes
print(f'{node_population.name}: {type(node_population)}')

ids = spike_pop.node_ids
node_population.get(ids, properties=['layer','synapse_class','x','y','z']).head()

In [ ]:
filtered = spikes.filter(group={'layer':'6'}, t_start=spikes.time_start, t_stop=spikes.time_stop)
filtered.report.head()

In [ ]:
filtered.raster();

In [ ]:
filtered.firing_rate_histogram();

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

ax = plt.gca()
ax.set_xlabel("Time [ms]")
ax.set_ylabel("PSTH [Hz]")
ax.set_title(f"PSTH for group: {filtered.group}")

times = filtered.report.index

time_start = np.min(times)
time_stop = np.max(times)

# heuristic for a nice bin size (~100 spikes per bin on average)
time_binsize = min(50.0, (time_stop - time_start) / ((len(times) / 100.0) + 1.0))

bins = np.append(np.arange(time_start, time_stop, time_binsize), time_stop)
hist, bin_edges = np.histogram(times, bins=bins)
node_count = len(snap_simulation.circuit.nodes.ids(filtered.group))  # Get length of node ids for whole `group`
freq = 1.0 * hist / node_count / (0.001 * time_binsize)

# use the middle of the bins instead of the start of the bin
ax.plot(0.5 * (bin_edges[1:] + bin_edges[:-1]), freq, label="PSTH", drawstyle="steps-mid");

In [ ]:
spikes.filter().raster();

In [ ]:
spikes.filter().raster(y_axis='etype');

In [ ]:
spikes.filter().isi(binsize=100);

### Soma Report Analysis

Let's [load](https://github.com/openbraininstitute/snap/blob/master/doc/source/notebooks/07_frame_reports.ipynb) the soma report using BluePySnap and plot it.

In [ ]:
snap_simulation.reports

In [ ]:
soma_report = snap_simulation.reports['SomaVoltRec']

In [ ]:
print(
    soma_report.time_start,
    soma_report.time_stop,
    soma_report.dt,
    soma_report.time_units
)  # Gives a warning in case the dt differs from simulation.dt

In [ ]:
soma_report.population_names

In [ ]:
soma_pop = soma_report['S1nonbarrel_neurons']
print(type(soma_pop))

In [ ]:
node_population = soma_pop.nodes
print(f'{node_population.name}: {type(node_population)}')

In [ ]:
ids = soma_pop.node_ids
node_population.get(ids, properties=['layer','synapse_class','x','y','z']).head()

In [ ]:
filtered = soma_report.filter(group={'layer':'6'}, t_start= soma_report.time_start, t_stop= soma_report.time_stop)
filtered.report.head()

In [ ]:
filtered.trace();

In [ ]:
soma_report.filter().trace();